In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

58889256/58889256 [==============================] - 0s 0us/step


In [3]:
base_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [4]:

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 413s 6s/step - loss: 1.8150 - accuracy: 0.4589 - val_loss: 1.1930 - val_accuracy: 0.5642
Epoch 2/50
65/65 [==============================] - 414s 6s/step - loss: 0.8374 - accuracy: 0.7114 - val_loss: 1.0567 - val_accuracy: 0.6615
Epoch 3/50
65/65 [==============================] - 412s 6s/step - loss: 0.5254 - accuracy: 0.8165 - val_loss: 1.0082 - val_accuracy: 0.6887
Epoch 4/50
65/65 [==============================] - 418s 6s/step - loss: 0.3251 - accuracy: 0.8856 - val_loss: 1.0503 - val_accuracy: 0.7023
Epoch 5/50
65/65 [==============================] - 411s 6s/step - loss: 0.2302 - accuracy: 0.9285 - val_loss: 1.0380 - val_accuracy: 0.6907
Epoch 6/50
65/65 [==============================] - 413s 6s/step - loss: 0.1893 - accuracy: 0.9348 - val_loss: 1.2128 - val_accuracy: 0.6809
Epoch 7/50
65/65 [==============================] - 413s 6s/step - loss: 0.1239 - accuracy: 0.9582 - val_loss: 1.2574 - val_accuracy: 0.6829
Epoch 8/50
65

In [8]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 177s 5s/step - loss: 0.2184 - accuracy: 0.9328
Accuracy: 0.9327883720397949
